<a href="https://colab.research.google.com/github/injilashah/TSD/blob/master/dynaic_ts_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dynamicts.data_loader import DataLoader

In [ ]:

loader = DataLoader(filepath="I:/CQAI/TSA/TSD/TSD/data/natural_gas_price.csv", index_col="Date")

In [ ]:
data =loader.load()

In [ ]:
loader.is_regular()

2025-07-06 23:24:56,415 - WARNING - Data is not regular. Index differences are not uniform.
2025-07-06 23:24:56,418 - WARNING - Unique differences found: <TimedeltaArray>
['1 days', '3 days', '4 days', '5 days', '2 days', '15 days']
Length: 6, dtype: timedelta64[ns]


False

In [ ]:

# Create complete date range from min to max
full_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')

# Find missing dates
missing_dates = full_range.difference(data.index)

print("Missing days:", len(missing_dates))
print(missing_dates)


Missing days: 2686
DatetimeIndex(['1997-01-11', '1997-01-12', '1997-01-18', '1997-01-19',
               '1997-01-25', '1997-01-26', '1997-02-01', '1997-02-02',
               '1997-02-08', '1997-02-09',
               ...
               '2020-08-01', '2020-08-02', '2020-08-08', '2020-08-09',
               '2020-08-15', '2020-08-16', '2020-08-22', '2020-08-23',
               '2020-08-29', '2020-08-30'],
              dtype='datetime64[ns]', length=2686, freq=None)


In [ ]:
diffs = data.index.to_series().diff().dropna()

In [ ]:
diffs

date
1997-01-08   1 days
1997-01-09   1 days
1997-01-10   1 days
1997-01-13   3 days
1997-01-14   1 days
              ...  
2020-08-26   1 days
2020-08-27   1 days
2020-08-28   1 days
2020-08-31   3 days
2020-09-01   1 days
Name: date, Length: 5952, dtype: timedelta64[ns]

In [ ]:



if diffs.nunique() == 1:
    print(f"Data is regular. Index differences are uniform: {diffs.iloc[0]}")

else:
    print("Data is not regular. Index differences are not uniform.")
    print(f"Unique differences found: {diffs.unique()}" )

In [ ]:
import pandas as pd
from collections import defaultdict


#Result dictionary
grouped_dates = defaultdict(list)


for date, delta in diffs.items():
    #if delta != pd.Timedelta(days=1):  # Skip 1-day diffs
    grouped_dates[str(delta)].append(str(date.date()))  # Convert Timestamp to string

#Convert to  dictionary
irregular_gaps = dict(grouped_dates)


print("✅ Irregular gaps stored in dictionary")
print(irregular_gaps)


✅ Irregular gaps stored in dictionary


Actual implementation

In [ ]:
print("Inferred frequency:", data.index.freq)

In [ ]:
from collections import defaultdict
import pandas as pd

def get_initial_unique_gaps(diffs):
    grouped_dates = defaultdict(list)
    for date, delta in diffs.items():
        #if delta != base_delta:
        grouped_dates[str(delta)].append(str(date.date()))
    return dict(grouped_dates)


In [ ]:
def compute_diffs_and_group(dates_list,iteration,key):
    timestamps = pd.to_datetime(dates_list)
    diffs = timestamps.to_series().diff().dropna()
    print(f"Iteration {iteration} Key {key} Unique differences:", diffs.unique())
    grouped = defaultdict(list)
    for date, delta in diffs.items():
        grouped[str(delta)].append(str(date.date()))
    return grouped


In [ ]:
from collections import defaultdict

def iterative_pattern_discovery(irregular_gaps, data):
    current_groups = irregular_gaps.copy()
    final_discarded = defaultdict(list)
    final_kept = defaultdict(list)
    iteration = 1

    while current_groups:
        next_groups = {}

        for delta_key, dates_list in current_groups.items():
            grouped = compute_diffs_and_group(dates_list)

            for new_delta, new_dates in grouped.items():
                count = len(new_dates)
                key = f"{delta_key} → {new_delta}"

                if count > 1:
                    next_groups[key] = new_dates
                    final_kept[key].extend(new_dates)
                else:
                    final_discarded[key].extend(new_dates)

        current_groups = next_groups
        iteration += 1

    # Count unique discarded dates
    discarded_dates = set(date for dates in final_discarded.values() for date in dates)
    total_discarded = len(discarded_dates)

    print(f"\nTotal entries in dataset: {len(data)}")
    print(f"Total discarded unique dates: {total_discarded}")
    print(f"Percentage of discarded entries: {(total_discarded / len(data)) * 100:.2f}%")

    return dict(final_kept), dict(final_discarded)


In [ ]:
# Assume df is your DataFrame with a datetime index
diffs = data.index.to_series().diff().dropna()


#Get initial irregular gaps
irregular_gaps = get_initial_unique_gaps(diffs)
print("Initial Unique differences:", diffs.unique())


Initial Unique differences: <TimedeltaArray>
['1 days', '3 days', '4 days', '5 days', '2 days', '15 days']
Length: 6, dtype: timedelta64[ns]


In [ ]:
from collections import defaultdict
import json

def get_dates_by_unique_diffs(data):
    diffs = data.index.to_series().diff().dropna()
    delta_to_dates = defaultdict(list)

    for i, delta in enumerate(diffs):
        current_date = data.index[i]  # i (not i+1) because diff aligns with the current row
        delta_to_dates[str(delta)].append(current_date.strftime('%Y-%m-%d'))

    # Save to JSON
    with open("unique_deltas_with_dates.json", "w") as f:
        json.dump(delta_to_dates, f, indent=4)

    return delta_to_dates
get_dates_by_unique_diffs(data)


In [ ]:
#Run iterative analysis
patterns, no_patterns = iterative_pattern_discovery(irregular_gaps,data)


import json
with open("kept_patterns.json", "w") as f1, open("discarded_patterns.json", "w") as f2:
    json.dump(patterns, f1,  indent=4)
    json.dump(no_patterns, f2, indent=4)


Iteration 1 Key 1 Unique differences: <TimedeltaArray>
['1 days', '4 days', '5 days', '6 days', '3 days', '19 days']
Length: 6, dtype: timedelta64[ns]
Iteration 1 Key 2 Unique differences: <TimedeltaArray>
['7 days', '14 days', '21 days', '42 days']
Length: 4, dtype: timedelta64[ns]
Iteration 1 Key 3 Unique differences: <TimedeltaArray>
[ '41 days',  '57 days', '140 days',  '28 days',  '55 days',  '43 days',
  '64 days', '111 days',   '7 days',  '15 days',  '48 days',  '35 days',
  '63 days',  '22 days',  '20 days',  '42 days',  '36 days',  '98 days',
 '112 days',  '14 days',  '62 days',  '50 days',  '13 days',  '34 days',
 '945 days',  '70 days']
Length: 26, dtype: timedelta64[ns]
Iteration 1 Key 4 Unique differences: <TimedeltaArray>
[ '364 days',   '36 days',  '328 days',   '30 days',  '194 days',  '147 days',
   '28 days',    '7 days',  '329 days',  '219 days',  '145 days', '4382 days',
  '187 days']
Length: 13, dtype: timedelta64[ns]
Iteration 1 Key 5 Unique differences: <Timedelt

In [ ]:
from collections import defaultdict

def iterative_pattern_discovery(irregular_gaps, data):
    current_groups = irregular_gaps.copy()
    final_discarded = defaultdict(list)
    final_kept = defaultdict(list)
    classified_dates = set()  # Track already classified dates
    iteration = 1

    while current_groups:
        next_groups = {}
        which_key = 1

        for delta_key, dates_list in current_groups.items():
            grouped = compute_diffs_and_group(dates_list,iteration,which_key)

            for new_delta, new_dates in grouped.items():
                # Remove dates that have already been classified
                new_dates = [d for d in new_dates if d not in classified_dates]
                if not new_dates:
                    continue

                count = len(new_dates)
                key = f"{delta_key} → {new_delta}"

                if count > 1:
                    next_groups[key] = new_dates
                    final_kept[key].extend(new_dates)
                else:
                    final_discarded[key].extend(new_dates)

                # Mark these dates as classified (either kept or discarded)
                classified_dates.update(new_dates)
            which_key +=1
        current_groups = next_groups
        iteration += 1

    # Count unique discarded dates
    discarded_dates = set(date for dates in final_discarded.values() for date in dates)
    total_discarded = len(discarded_dates)

    print(f"\nTotal entries in dataset: {len(data)}")
    print(f"Total discarded unique dates: {total_discarded}")
    print(f"Percentage of discarded entries: {(total_discarded / len(data)) * 100:.2f}%")

    return dict(final_kept), dict(final_discarded)


In [ ]:
data.index

DatetimeIndex(['1997-01-07', '1997-01-08', '1997-01-09', '1997-01-10',
               '1997-01-13', '1997-01-14', '1997-01-15', '1997-01-16',
               '1997-01-17', '1997-01-20',
               ...
               '2020-08-19', '2020-08-20', '2020-08-21', '2020-08-24',
               '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28',
               '2020-08-31', '2020-09-01'],
              dtype='datetime64[ns]', name='date', length=5953, freq=None)

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

def analyze_date_regularity(dates):
    """Optimized version of date regularity analysis."""
    # Preprocessing with vectorized operations
    dates = pd.DatetimeIndex(dates).dropna().unique().sort_values()
    if len(dates) < 2:
        return {
            'kept_dates': dates,
            'discarded_dates': pd.DatetimeIndex([]),
            'is_regular': True,
            'hierarchy_tree': {},
            'iterations': []
        }

    # Initialize data structures
    keep = np.ones(len(dates), dtype=bool)
    date_to_index = {date: i for i, date in enumerate(dates)}
    hierarchy_tree = defaultdict(list)
    iterations = []
    node_counter = 0

    def process_node(date_indices, parent_node=None, level=0, gap_value=None):
        nonlocal node_counter, keep

        if len(date_indices) < 2:
            return

        # Vectorized difference calculation
        diffs = np.diff(dates[date_indices]) / np.timedelta64(1, 'D')
        unique_diffs, counts = np.unique(diffs, return_counts=True)

        # Create node
        current_node = node_counter
        node_counter += 1
        if parent_node is not None:
            hierarchy_tree[parent_node].append((current_node, gap_value))

        # Store iteration info
        iterations.append({
            'level': level,
            'node_id': current_node,
            'gap_value': gap_value,
            'dates_count': len(date_indices),
            'unique_gaps': unique_diffs[counts == 1].tolist(),
            'non_unique_gaps': unique_diffs[counts > 1].tolist()
        })

        # Process unique gaps (to discard)
        for gap in unique_diffs[counts == 1]:
            pos = np.where(diffs == gap)[0][0]
            discard_idx = date_indices[pos + 1]
            keep[discard_idx] = False

        # Process non-unique gaps (to recurse)
        for gap in unique_diffs[counts > 1]:
            gap_positions = np.where(diffs == gap)[0]
            child_indices = set()
            for pos in gap_positions:
                child_indices.add(date_indices[pos])
                child_indices.add(date_indices[pos + 1])
            process_node(
                sorted(child_indices),
                current_node,
                level + 1,
                gap
            )

    # Start processing
    process_node(np.arange(len(dates)))

    # Prepare results
    kept_dates = dates[keep]
    discarded_dates = dates[~keep]
    perc_discarded = len(discarded_dates) / len(dates) * 100

    return {
        'kept_dates': kept_dates,
        'discarded_dates': discarded_dates,
        'total_dates': len(dates),
        'kept_count': len(kept_dates),
        'discarded_count': len(discarded_dates),
        'discarded_percentage': perc_discarded,
        'iterations': iterations,
        'hierarchy_tree': dict(hierarchy_tree),
        'is_regular': perc_discarded <= 50
    }

# Example Usage:
if __name__ == "__main__":
    # Create sample data with irregular gaps


    result = analyze_date_regularity(data.index)

    print(f"Total Dates: {result['total_dates']}")
    print(f"Kept Dates: {result['kept_count']}")
    print(f"Discarded Dates: {result['discarded_count']} ({result['discarded_percentage']:.2f}%)")
    print(f"Data is Regular: {result['is_regular']}")

    # Print iteration hierarchy
    print("\nHierarchy Tree (Parent -> Children with Gap Value):")
    for parent, children in result['hierarchy_tree'].items():
        child_ids = [f"{child_id} (gap={gap_val})" for child_id, gap_val in children]
        print(f"Node {parent} -> {', '.join(child_ids)}")

    # Print iterations summary
    print("\nIteration Details:")
    for i, iter_info in enumerate(result['iterations']):
        print(f"\nIteration {i} (Level {iter_info['level']}):")
        print(f"  Node {iter_info['node_id']} - Gap Value: {iter_info['gap_value']}")
        print(f"  Dates in Node: {iter_info['dates_count']}")
        print(f"  Unique Gaps: {iter_info['unique_gaps']} (Discarded {len(iter_info['discarded_in_node'])} dates)")
        print(f"  Non-Unique Gaps: {iter_info['non_unique_gaps']}")

RecursionError: maximum recursion depth exceeded

In [ ]:
import numpy as np
import pandas as pd
from collections import deque  # For efficient stack operations

def analyze_date_regularities(date_series, min_keep_ratio=0.5):
    """Iterative version to avoid recursion limits."""
    # Input validation
    if not isinstance(date_series.index, pd.DatetimeIndex):
        raise ValueError("Input must have a DatetimeIndex.")

    dates = date_series.index.sort_values().unique()
    if len(dates) < 2:
        return {
            "is_regular": True,
            "total_dates": len(dates),
            "num_kept": len(dates),
            "num_discarded": 0,
            "discard_ratio": 0.0,
            "hierarchy": {}
        }

    dates_np = dates.to_numpy()
    keep_mask = np.ones(len(dates_np), dtype=bool)
    hierarchy = {}
    stack = deque()

    # Initialize stack with root node
    stack.append((np.arange(len(dates_np)), [], "root"))

    while stack:
        date_indices, level_path, parent_key = stack.pop()

        if len(date_indices) < 2:
            continue

        # Calculate gaps
        days_diff = (dates_np[date_indices[1:]] - dates_np[date_indices[:-1]]).astype('timedelta64[D]').astype(int)
        unique_diffs, counts = np.unique(days_diff, return_counts=True)

        for delta, count in zip(unique_diffs, counts):
            path_key = level_path + [f"{delta}_days"]
            full_key = " -> ".join(path_key)

            if count == 1:
                # Discard end date of unique gap
                gap_pos = np.where(days_diff == delta)[0][0]
                discard_idx = date_indices[gap_pos + 1]
                keep_mask[discard_idx] = False
                hierarchy[full_key] = {"count": 1, "delta": f"{delta}_days", "action": "discarded"}
            else:
                # Schedule subgroup for processing
                gap_indices = np.where(days_diff == delta)[0]
                subgroup_indices = np.unique(np.concatenate([
                    date_indices[gap_indices],
                    date_indices[gap_indices + 1]
                ]))
                hierarchy[full_key] = {"count": len(subgroup_indices), "delta": f"{delta}_days"}
                stack.append((subgroup_indices, path_key, full_key))

    # Compile results
    num_discarded = len(dates_np) - np.sum(keep_mask)
    discard_ratio = num_discarded / len(dates_np)
    is_regular = discard_ratio <= (1 - min_keep_ratio)

    return {
        "is_regular": is_regular,
        "total_dates": len(dates_np),
        "num_kept": np.sum(keep_mask),
        "num_discarded": num_discarded,
        "discard_ratio": discard_ratio,
        "min_keep_ratio": min_keep_ratio,
        "hierarchy": hierarchy
    }

In [ ]:
# Ensure the index is named 'date' for compatibility
data.index.name = 'date'
result = analyze_date_regularities(pd.Series(index=data.index))

# Output
print(json.dumps(result, indent=2))
print(f"Kept: {result['num_kept']}, Discarded: {result['num_discarded']}")
print(f"Hierarchy keys: {list(result['hierarchy'].keys())[:5]}")  # Show first 5 patterns


In [ ]:
data

,price
date,
1997-01-07,3.82
1997-01-08,3.80
1997-01-09,3.61
1997-01-10,3.92
1997-01-13,4.00
...,...
2020-08-26,2.52
2020-08-27,2.52
2020-08-28,2.46


In [ ]:
data.index

DatetimeIndex(['1997-01-07', '1997-01-08', '1997-01-09', '1997-01-10',
               '1997-01-13', '1997-01-14', '1997-01-15', '1997-01-16',
               '1997-01-17', '1997-01-20',
               ...
               '2020-08-19', '2020-08-20', '2020-08-21', '2020-08-24',
               '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28',
               '2020-08-31', '2020-09-01'],
              dtype='datetime64[ns]', name='date', length=5953, freq=None)